In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
    return np.abs(np.log(
        (responses_column + epsilon)
        / (answers_column + epsilon)
    )).mean()


def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):  # answers - правильные ответы
    log_accuracy_ratio_mean = np.array(
        [
            get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon),
        ]
    ).mean()

    percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error.round(
        decimals=2
    )

In [0]:
import pandas as pd
import numpy as np

In [0]:
DF = "drive/My Drive/VK_Cup_#2/Data/top_validate.tsv"
VALIDATE_ANSWERS = "drive/My Drive/VK_Cup_#2/Data/validate_answers.tsv"

In [0]:
Y = pd.read_csv(VALIDATE_ANSWERS, sep='\t')

In [741]:
Y.head()

,at_least_one,at_least_two,at_least_three
0,0.0430,0.0152,0.0073
1,0.0130,0.0000,0.0000
2,0.0878,0.0135,0.0000
3,0.2295,0.1295,0.0727
4,0.3963,0.2785,0.2270


In [0]:
Y_1 = Y['at_least_one'].copy()
Y_2 = Y['at_least_two'].copy()
Y_3 = Y['at_least_three'].copy()

In [743]:
Y_1

0       0.0430
1       0.0130
2       0.0878
3       0.2295
4       0.3963
         ...  
1003    0.0679
1004    0.0165
1005    0.0071
1006    0.3669
1007    0.0710
Name: at_least_one, Length: 1008, dtype: float64

In [0]:
X = pd.read_csv(DF).astype('float32')

In [745]:
X.columns

Index(['cpm', 'hour_start', 'hour_end', 'audience_size', 'time_interval',
       'hour_start_time', 'hour_end_time', 'time_interval_time', 'middle_time',
       'part_day', 'people_partday_platform',
       'percent_people_partday_platform', 'people_patday',
       'percent_people_patday', 'percent_people_partday_platformy',
       'count_unique_cities', 'number_of_sex1', 'number_of_sex2',
       'number_of_age1', 'number_of_age2', 'number_of_age3', 'number_of_age4',
       'number_of_age5', 'number_of_age6', 'percentage_sex1',
       'percentage_sex2', 'percentage_age1', 'percentage_age2',
       'percentage_age3', 'percentage_age4', 'percentage_age5',
       'percentage_age6', 'mean_cpm_in_partday_and_publishers',
       'number_of_publishers', 'publisher_1', 'publisher_2', 'publisher_3',
       'publisher_4', 'publisher_5', 'publisher_6', 'publisher_7',
       'publisher_8', 'publisher_9', 'publisher_10', 'publisher_11',
       'publisher_12', 'publisher_13', 'publisher_14', 'publis

In [0]:
X = X.drop(['hour_start', 'hour_end', 'number_of_sex1', 'part_day', 'number_of_sex2', 'percent_people_partday_platform', 'percentage_sex1', 'percentage_sex2', 'number_who_use_that_publishers', 'count_unique_cities', 'number_of_publishers', 'number_of_age1', 'number_of_age2', 'number_of_age3', 'number_of_age4', 'number_of_age5', 'number_of_age6', 'middle_time'], axis=1)

In [747]:
X.columns

Index(['cpm', 'audience_size', 'time_interval', 'hour_start_time',
       'hour_end_time', 'time_interval_time', 'people_partday_platform',
       'people_patday', 'percent_people_patday',
       'percent_people_partday_platformy', 'percentage_age1',
       'percentage_age2', 'percentage_age3', 'percentage_age4',
       'percentage_age5', 'percentage_age6',
       'mean_cpm_in_partday_and_publishers', 'publisher_1', 'publisher_2',
       'publisher_3', 'publisher_4', 'publisher_5', 'publisher_6',
       'publisher_7', 'publisher_8', 'publisher_9', 'publisher_10',
       'publisher_11', 'publisher_12', 'publisher_13', 'publisher_14',
       'publisher_15', 'publisher_16', 'publisher_17', 'publisher_18',
       'publisher_19', 'publisher_20', 'publisher_21',
       'percentage_publishers_users'],
      dtype='object')

In [0]:
x_lim=int(len(X)*0.80)

X_train=X[0:x_lim].copy()
Y_train_1=Y_1[0:x_lim]
Y_train_2=Y_2[0:x_lim]
Y_train_3=Y_3[0:x_lim]

X_test=X[x_lim:].copy()
Y_test_1=Y_1[x_lim:]
Y_test_2=Y_2[x_lim:]
Y_test_3=Y_3[x_lim:]

In [749]:
Y_train_1.shape, Y_test_1.shape

((806,), (202,))

In [0]:
Y_test = pd.concat([Y_test_1, Y_test_2, Y_test_3], axis=1)
Y_test = pd.DataFrame(np.array(Y_test), columns=['at_least_one', 'at_least_two', 'at_least_three']).astype('float32')

In [751]:
Y_test.head()

,at_least_one,at_least_two,at_least_three
0,0.0163,0.0041,0.0014
1,0.0201,0.0000,0.0000
2,0.1761,0.0954,0.0578
3,0.0211,0.0105,0.0039
4,0.1165,0.0059,0.0000


In [752]:
from sklearn.preprocessing import MinMaxScaler

mms=MinMaxScaler()
X_train_mms=mms.fit_transform(X_train)
X_test_mms=mms.fit_transform(X_test)

print('Минимум и максимум обучающего набора: ', X_train_mms.min(), X_train_mms.max())

Минимум и максимум обучающего набора:  0.0 1.0


In [753]:
from sklearn.preprocessing import StandardScaler

mms=StandardScaler()
X_train_mms=mms.fit_transform(X_train)
X_test_mms=mms.fit_transform(X_test)

print('Минимум и максимум обучающего набора: ', X_train_mms.min(), X_train_mms.max())

Минимум и максимум обучающего набора:  -3.825924 6.225884


In [0]:
X_train_mms = X_train
X_test_mms = X_test

In [0]:
import xgboost as xgb

In [774]:
xgb_model1 = xgb.XGBRegressor(booster = 'gbtree', max_depth = 2, n_estimators = 7000, objective = 'binary:logistic', random_state=42)
xgb_model2 = xgb.XGBRegressor(booster = 'gbtree', max_depth = 2, n_estimators = 7000, objective = 'binary:logistic', random_state=42)
xgb_model3 = xgb.XGBRegressor(booster = 'gbtree', max_depth = 2, n_estimators = 7000, objective = 'binary:logistic', random_state=42)

xgb_model1.fit(X_train_mms, Y_train_1)
predictions_1 = xgb_model1.predict(X_test_mms)
xgb_model2.fit(X_train_mms, Y_train_2)
predictions_2 = xgb_model2.predict(X_test_mms)
xgb_model3.fit(X_train_mms, Y_train_3)
predictions_3 = xgb_model3.predict(X_test_mms)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [0]:
Y_pesponse = pd.DataFrame([predictions_1, predictions_2, predictions_3]).astype('float32').T
Y_pesponse.columns=['at_least_one', 'at_least_two', 'at_least_three']

In [776]:
get_smoothed_mean_log_accuracy_ratio(Y_test, Y_pesponse)

23.42

In [0]:
23.42 7000
23.48 6000
23.63 5000
23.93 4000
24.43 3000

24.53 - X.drop(['hour_start', 'hour_end', 'middle_time'], axis=1)
24.75 - X.drop(['hour_start', 'hour_end', 'number_of_age1', 'number_of_age2', 'number_of_age3', 'number_of_age4', 'number_of_age5', 'number_of_age6'], axis=1)
24.79 - X.drop(['hour_start', 'hour_end', 'number_of_publishers'], axis=1)
24.83 - X.drop(['hour_start', 'hour_end', 'count_unique_cities'], axis=1)
24.85 - X.drop(['hour_start', 'hour_end', 'number_who_use_that_publishers'], axis=1)
24.9 - X.drop(['hour_start', 'hour_end', 'percentage_sex1', 'percentage_sex2'], axis=1)
24.94 - X.drop(['hour_start', 'hour_end', 'percent_people_partday_platform'], axis=1)
24.94 - X.drop(['hour_start', 'hour_end', 'number_of_sex1', 'number_of_sex2'], axis=1)
24.95 - X.drop(['hour_start', 'hour_end'], axis=1)

X.drop(['hour_start', 'hour_end', 'number_of_sex1', 'part_day', 'number_of_sex2', 'percent_people_partday_platform', 'percentage_sex1', 'percentage_sex2', 'number_who_use_that_publishers', 'count_unique_cities', 'number_of_publishers', 'number_of_age1', 'number_of_age2', 'number_of_age3', 'number_of_age4', 'number_of_age5', 'number_of_age6', 'middle_time'], axis=1)



In [0]:
31.6 - X.drop(['hour_start', 'hour_end','middle_time', 'part_day', 'people_partday_platform', 'count_unique_cities'], axis=1)
32.07 - X.drop(['hour_start', 'hour_end','middle_time', 'part_day', 'people_partday_platform'], axis=1)
32.77 - X.drop(['hour_start', 'hour_end','middle_time', 'part_day'], axis=1)

In [0]:
21.82 - 8000 - X.drop(['hour_start', 'hour_end', 'number_of_sex1', 'number_of_sex2', 'number_of_age1', 'number_of_age2', 'number_of_age3', 'number_of_age4', 'number_of_age5', 'number_of_age6', 'number_who_use_that_publishers', 'percentage_publishers_users', 'audience_size', 'middle_time'], axis=1)
21.95 - 7000
22.13 - 6000
22.42 - 5000
22.61 - 4500

25.87 - 4500

26.19 - X.drop(['hour_start', 'hour_end', 'number_of_sex1', 'number_of_sex2', 'number_of_age1', 'number_of_age2', 'number_of_age3',
       'number_of_age4', 'number_of_age5', 'number_of_age6', 'number_who_use_that_publishers', 'mean_cpm_in_partday_and_publishers','percentage_publishers_users', 'audience_size', 'middle_time'], axis=1)

26.62 - X.drop(['hour_start', 'hour_end', 'number_of_sex1', 'number_of_sex2', 'number_of_age1', 'number_of_age2', 'number_of_age3',
       'number_of_age4', 'number_of_age5', 'number_of_age6', 'number_who_use_that_publishers', 'mean_cpm_in_partday_and_publishers','percentage_publishers_users', 'audience_size'], axis=1)

26.68 - X.drop(['hour_start', 'hour_end', 'number_of_sex1', 'number_of_sex2', 'number_of_age1', 'number_of_age2', 'number_of_age3',
       'number_of_age4', 'number_of_age5', 'number_of_age6', 'number_who_use_that_publishers', 'mean_cpm_in_partday_and_publishers','percentage_publishers_users'], axis=1)

26.8 - X.drop(['hour_start', 'hour_end', 'number_of_sex1', 'number_of_sex2', 'number_of_age1', 'number_of_age2', 'number_of_age3',
       'number_of_age4', 'number_of_age5', 'number_of_age6', 'number_who_use_that_publishers', 'mean_cpm_in_partday_and_publishers'], axis=1)

27.1 - X.drop(['hour_start', 'hour_end', 'number_of_sex1', 'number_of_sex2', 'number_of_age1', 'number_of_age2', 'number_of_age3',
       'number_of_age4', 'number_of_age5', 'number_of_age6', 'number_who_use_that_publishers'], axis=1)

27.26 - X.drop(['hour_start', 'hour_end', 'number_of_sex1', 'number_of_sex2', 'number_of_age1', 'number_of_age2', 'number_of_age3',
       'number_of_age4', 'number_of_age5', 'number_of_age6'], axis=1)

In [0]:
23.18

In [777]:
xgb_model1 = xgb.XGBRegressor(booster = 'gbtree', max_depth = 2, n_estimators = 7000, objective = 'binary:logistic', random_state=42)
xgb_model2 = xgb.XGBRegressor(booster = 'gbtree', max_depth = 2, n_estimators = 7000, objective = 'binary:logistic', random_state=42)
xgb_model3 = xgb.XGBRegressor(booster = 'gbtree', max_depth = 2, n_estimators = 7000, objective = 'binary:logistic', random_state=42)

xgb_model1.fit(X, Y_1)
#predictions_1 = er1.predict(X_test_mms)
xgb_model2.fit(X, Y_2)
#predictions_2 = er2.predict(X_test_mms)
xgb_model3.fit(X, Y_3)
#predictions_3 = er3.predict(X_test_mms)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=7000,
             n_jobs=1, nthread=None, objective='binary:logistic',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [778]:
import joblib

joblib.dump(xgb_model1, 'drive/My Drive/VK_Cup_#2/models/xgb_model1_2342.model')
joblib.dump(xgb_model2, 'drive/My Drive/VK_Cup_#2/models/xgb_model2_2342.model')
joblib.dump(xgb_model3, 'drive/My Drive/VK_Cup_#2/models/xgb_model3_2342.model')

['drive/My Drive/VK_Cup_#2/models/xgb_model3_2342.model']